# Guided Proejct: Mobile App for Lottery Addiction

Aim of this notebook is to build a set of tools to help users calculate the probability of winning the lottery under various conditions, so they can see their odds to help break addiction. We'll be simulating the following lottery https://en.wikipedia.org/wiki/Lotto_6/49

In [5]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import math

In [2]:
def factorial(n):
    return math.factorial(n)


def combinations(n, k):
    return math.comb(n, k)


In [3]:
def one_ticket_probability(user_numbers):
    total_outcomes = combinations(49, 6)
    probability = 1 / total_outcomes
    percentage_form = probability * 100
    print('''Your chances to win the big prize with the numbers {} are {:.7f}%.
In other words, you have a 1 in {:,} chances to win.'''.format(user_numbers,
                                                               percentage_form, int(total_outcomes)
                                                               )
          )

In [6]:
numbers_1 = [2, 5, 13, 24, 25, 36]
one_ticket_probability(numbers_1)

Your chances to win the big prize with the numbers [2, 5, 13, 24, 25, 36] are 0.0000072%.
In other words, you have a 1 in 13,983,816 chances to win.


In [8]:
lottery_results = pd.read_csv('649.csv')
lottery_results.head()


,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
0,649,1,0,6/12/1982,3,11,12,14,41,43,13
1,649,2,0,6/19/1982,8,33,36,37,39,41,9
2,649,3,0,6/26/1982,1,6,23,24,27,39,34
3,649,4,0,7/3/1982,3,9,10,13,20,43,34
4,649,5,0,7/10/1982,5,14,21,31,34,47,45


In [9]:
lottery_results.tail()

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
3660,649,3587,0,6/6/2018,10,15,23,38,40,41,35
3661,649,3588,0,6/9/2018,19,25,31,36,46,47,26
3662,649,3589,0,6/13/2018,6,22,24,31,32,34,16
3663,649,3590,0,6/16/2018,2,15,21,31,38,49,8
3664,649,3591,0,6/20/2018,14,24,31,35,37,48,17


The engineering team told us that we need to be aware of the following details:

Inside the app, the user inputs six different numbers from 1 to 49.
Under the hood, the six numbers will come as a Python list and serve as an input to our function.
The engineering team wants us to write a function that prints:
the number of times the combination selected occurred in the Canada data set; and
the probability of winning the big prize in the next drawing with that combination.
We'll now start working on writing this function. Note there's more than one way to solve this problem, so take the instructions below as suggestions.

In [10]:
# Step 1: Define a function to extract the six winning numbers from a row as a set
def extract_numbers(row):
    return {row['NUMBER DRAWN 1'], row['NUMBER DRAWN 2'], row['NUMBER DRAWN 3'],
            row['NUMBER DRAWN 4'], row['NUMBER DRAWN 5'], row['NUMBER DRAWN 6']}


# Step 2: Apply the function to the DataFrame to extract all winning numbers
lottery_results['winning_numbers_set'] = lottery_results.apply(extract_numbers, axis=1)


# Step 3: Define a function to check historical occurrence of a user's number combination
def check_historical_occurrence(user_numbers, winning_numbers_series):
    user_numbers_set = set(user_numbers)
    matches = winning_numbers_series.apply(lambda x: x == user_numbers_set)

    # Output the number of matches and calculate probability
    occurrence_count = matches.sum()
    total_draws = len(winning_numbers_series)
    probability = (1 / total_draws) if occurrence_count > 0 else 0

    print(f"The combination {user_numbers} occurred {occurrence_count} times in the past.")
    print(f"The probability of winning with this combination in the next draw is approximately {probability:.8f}")


# Example: Test the function with a sample user input
user_input = [3, 41, 11, 12, 43, 14]
check_historical_occurrence(user_input, lottery_results['winning_numbers_set'])


The combination [3, 41, 11, 12, 43, 14] occurred 1 times in the past.
The probability of winning with this combination in the next draw is approximately 0.00027285


In [11]:
# Step 2: Define the multi_ticket_probability function
def multi_ticket_probability(number_of_tickets):
    # Total number of combinations for a 6-number lottery ticket from 49 numbers
    total_outcomes = combinations(49, 6)

    # Successful outcomes equal the number of tickets played
    successful_outcomes = number_of_tickets

    # Calculate the probability
    probability = successful_outcomes / total_outcomes

    # Print the result as a percentage with explanation
    print("The probability of winning with {} ticket(s) is {:.10f}%.".format(number_of_tickets, probability * 100))


# Step 3: Test the function with the provided inputs
test_inputs = [1, 10, 100, 10000, 1000000, 6991908, 13983816]
for tickets in test_inputs:
    multi_ticket_probability(tickets)

The probability of winning with 1 ticket(s) is 0.0000071511%.
The probability of winning with 10 ticket(s) is 0.0000715112%.
The probability of winning with 100 ticket(s) is 0.0007151124%.
The probability of winning with 10000 ticket(s) is 0.0715112384%.
The probability of winning with 1000000 ticket(s) is 7.1511238420%.
The probability of winning with 6991908 ticket(s) is 50.0000000000%.
The probability of winning with 13983816 ticket(s) is 100.0000000000%.


In [12]:
# Step 1: Define the probability_less_6 function
def probability_less_6(matches):
    if matches < 2 or matches > 5:
        print("Please input an integer between 2 and 5.")
        return
    
    # Total number of combinations for a 6-number lottery ticket from 49 numbers
    total_outcomes = combinations(49, 6)
    
    # Number of successful outcomes depends on the number of matches
    # We need to combine the successful matches with the remaining unmatched numbers
    # For `matches` successful, we choose `matches` from 6, and the remaining from 43
    successful_outcomes = combinations(6, matches) * combinations(43, 6 - matches)
    
    # Calculate the probability
    probability = successful_outcomes / total_outcomes
    
    # Print the result as a percentage with explanation
    print("The probability of matching exactly {} number(s) is {:.10f}%.".format(matches, probability * 100))


# Step 2: Test the function with inputs: 2, 3, 4, 5
for match in range(2, 6):
    probability_less_6(match)


The probability of matching exactly 2 number(s) is 13.2378029002%.
The probability of matching exactly 3 number(s) is 1.7650403867%.
The probability of matching exactly 4 number(s) is 0.0968619724%.
The probability of matching exactly 5 number(s) is 0.0018449900%.
